In [34]:
from selenium import webdriver
import requests
import time
import pandas as pd
from IPython.display import clear_output

In [17]:
# 웹드라이브 실행
driver = webdriver.Chrome('chromedriver')

In [18]:
# 다음 영화 사이트를 들어간다.
site = 'https://movie.daum.net/premovie/theater'
time.sleep(1)
driver.get(site)

In [19]:
# 스크롤한다.
for idx in range(10):
    time.sleep(1)
    script = 'window.scrollTo(0, document.body.scrollHeight)'
    driver.execute_script(script)

In [20]:
# 영화 전체 목록을 가져온다.
a1 = driver.find_element_by_css_selector('#mainContent > div > div.box_movie > ol')
a1

<selenium.webdriver.remote.webelement.WebElement (session="1280aac9b11e58e3ba50a9f51e4494ad", element="e4f52d84-36cb-482a-8c03-432f687511c4")>

In [23]:
# 영화 전체 목록에서 li태그들을 가져온다.
li_list = a1.find_elements_by_css_selector('li')
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="1280aac9b11e58e3ba50a9f51e4494ad", element="ab8c5e3d-5351-4fe8-ad72-3b11e0e51348")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1280aac9b11e58e3ba50a9f51e4494ad", element="32dc3311-d4c3-4a83-9c09-69e3d8ed56db")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1280aac9b11e58e3ba50a9f51e4494ad", element="21c3af57-e205-4f3b-b8f9-38a787e9164d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1280aac9b11e58e3ba50a9f51e4494ad", element="63c83caf-6910-4448-8b7c-e282b06537b1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1280aac9b11e58e3ba50a9f51e4494ad", element="8aabf706-5005-45c0-9f3b-7963885570f9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1280aac9b11e58e3ba50a9f51e4494ad", element="4b2caf55-c73a-44fa-b713-fc4ca013426d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1280aac9b11e58e3ba50a9f51e4494ad", element="40362c66-f88b-40d0-8f52-0f

In [46]:
# 데이터를 담을 딕셔너리
data_dict = {
    '제목' : [],
    '예매율' : [],
    '평점' : [],
    '개봉일' : []
}

# 영화의 수 만큼 반복한다..
for movie_tag in li_list:
    # 영화 제목을 가져온다.
    title_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > strong > a')
    title = title_tag.text.strip()
    
    # 평점
    rating_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(1) > span')
    rating = rating_tag.text.strip()
    
    # 예매율
    ticket_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(2) > span')
    ticket = ticket_tag.text.strip()
    
    # 개봉
    open_date_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_info > span')
    open_date = open_date_tag.text.strip()
    
    # 영화 포스터
    poster_tag = movie_tag.find_element_by_css_selector('div > div.thumb_item > div.poster_movie > img')
    # src 속성의 값을 가져온다.
    src_attr = poster_tag.get_attribute('src')
    # print(src_attr)
    
    # 영화의 제목을 파일명으로 사용할 것이기 때문에 os에서 거부하는 파일명을 정제한다.
    char_list = ['\\', '/', ':', '*', '?', '"', '<', '>', '|']
    
    file_name = title
    
    for c1 in char_list:
        file_name = file_name.replace(c1, ' ')
    
    # 이미지 데이터를 내려받는다.
    img_res = requests.get(src_attr)
    
    # 저장한다.
    with open(f'poster/{file_name}.jpg', 'wb') as fp:
            fp.write(img_res.content)

    
    
#     print(f'제목: {title}')
#     print(f'평점: {rating}')
#     print(f'예매율: {ticket}')
#     print(f'개봉일: {release}')
#     print('-------------------------')

    
    # 데이터를 담는다.
    data_dict['제목'].append(title)
    data_dict['예매율'].append(ticket[:-1])
    data_dict['평점'].append(rating)
    data_dict['개봉일'].append(open_date)

# 데이터 프레임 생성
df1 = pd.DataFrame(data_dict)
df1

# 저장
df1.to_csv('daum_movie.csv', encoding='utf-8-sig', index=False)
print('저장완료')

저장완료


In [37]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      34 non-null     object
 1   예매율     34 non-null     object
 2   평점      34 non-null     object
 3   개봉일     34 non-null     object
dtypes: object(4)
memory usage: 1.2+ KB
